In [2]:
# Dependencies
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import sem
import os
from pathlib import Path
import requests
import json
from scipy.stats import linregress, pearsonr
import scipy.stats as st


In [ ]:
#load files
outbound_shipments_2018 = os.path.join(".", "resources", "outbound_shipments_2018.csv")
period_num = os.path.join(".", "resources", "period_numbers_2016to2019.csv")
states = os.path.join(".", "resources", "states.csv")

#read data
outbound_shipments_2018_df = pd.read_csv(outbound_shipments_2018)
period_num_df = pd.read_csv(period_num)
states_df = pd.read_csv(states)



In [4]:
#####2018 DATASET

 #merge outbound_shipments_2018 and period_num_df
combined_2018_outbound_period_df = pd.merge(
    outbound_shipments_2018_df,
    period_num_df[["date", "period_number"]],  
    left_on="Shipped Date",
    right_on="date",
    how="left").drop(columns="date")

combined_2018_outbound_period_df

#combined_outbound_period_df with states_df
combined_2018_dataset_df = pd.merge(
    combined_2018_outbound_period_df,
    states_df[["state_abr", "country"]],  
    left_on="Shipping State",
    right_on="state_abr",
    how="left").drop(columns="state_abr")

combined_2018_dataset_df

,Navision Order ID,Navision Created Date,Channel,Channel Order ID,Shipping Agent,Tracking No,Order Status,Status Summary,Shipping City,Shipping State,Shipped Date,Delivered On,Status,Delivered,period_number,country
0,SYMSP211039,1/1/18 0:00,ROYAL,32107548,FEDEX,9.74468E+14,-NONE-,NaN,HOLBROOK,NY,1/2/18,NaN,NaN,0,1,US
1,SYMSP211039,1/1/18 21:05,ROYAL CANIN USA,32107548,FEDEX,9.74468E+14,-NONE-,NaN,HOLBROOK,NY,1/2/18,NaN,NaN,0,1,US
2,SYMSP211132,1/2/18 17:15,ROYAL CANIN USA,32109579,FEDEX,9.74468E+14,-NONE-,NaN,NEW BRAUNFELS,TX,1/2/18,NaN,NaN,0,1,US
3,SYMSP211122,1/2/18 16:12,ROYAL CANIN USA,32109423,FEDEX,9.74468E+14,-NONE-,NaN,Frisco,CO,1/2/18,NaN,NaN,0,1,US
4,SYMSP211112,1/2/18 14:09,ROYAL CANIN USA,32108343,FEDEX,9.74468E+14,-NONE-,NaN,DAHLONEGA,GA,1/2/18,NaN,NaN,0,1,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67595,SYMSP276872,12/28/18 17:14,ROYAL CANIN USA,35136117,FEDEX,4.41128E+11,Shipped,Delivered,CHALFONT,PA,12/28/18,12/31/18,12/31/18 0:00,1,13,US
67596,SYMSP276873,12/28/18 17:14,ROYAL CANIN USA,35136337,FEDEX,4.41128E+11,Shipped,Delivered,COCOA,FL,12/28/18,1/2/19,1/2/19 0:00,1,13,US
67597,SYMSP276874,12/28/18 17:14,ROYAL CANIN USA,14082922,FEDEX,4.41128E+11,Shipped,Delivered,IRVINE,CA,12/28/18,1/4/19,1/4/19 0:00,1,13,US
67598,SYMSP276871,12/28/18 17:14,ROYAL CANIN USA,35134300,FEDEX,4.41128E+11,Shipped,Delivered,ASBURY PARK,NJ,12/28/18,1/2/19,1/2/19 0:00,1,13,US


In [5]:

combined_2018_dataset_df.dtypes

Navision Order ID        object
Navision Created Date    object
Channel                  object
Channel Order ID         object
Shipping Agent           object
Tracking No              object
Order Status             object
Status Summary           object
Shipping City            object
Shipping State           object
Shipped Date             object
Delivered On             object
Status                   object
Delivered                 int64
period_number             int64
country                  object
dtype: object

In [6]:
combined_2018_dataset_df

,Navision Order ID,Navision Created Date,Channel,Channel Order ID,Shipping Agent,Tracking No,Order Status,Status Summary,Shipping City,Shipping State,Shipped Date,Delivered On,Status,Delivered,period_number,country
0,SYMSP211039,1/1/18 0:00,ROYAL,32107548,FEDEX,9.74468E+14,-NONE-,NaN,HOLBROOK,NY,1/2/18,NaN,NaN,0,1,US
1,SYMSP211039,1/1/18 21:05,ROYAL CANIN USA,32107548,FEDEX,9.74468E+14,-NONE-,NaN,HOLBROOK,NY,1/2/18,NaN,NaN,0,1,US
2,SYMSP211132,1/2/18 17:15,ROYAL CANIN USA,32109579,FEDEX,9.74468E+14,-NONE-,NaN,NEW BRAUNFELS,TX,1/2/18,NaN,NaN,0,1,US
3,SYMSP211122,1/2/18 16:12,ROYAL CANIN USA,32109423,FEDEX,9.74468E+14,-NONE-,NaN,Frisco,CO,1/2/18,NaN,NaN,0,1,US
4,SYMSP211112,1/2/18 14:09,ROYAL CANIN USA,32108343,FEDEX,9.74468E+14,-NONE-,NaN,DAHLONEGA,GA,1/2/18,NaN,NaN,0,1,US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67595,SYMSP276872,12/28/18 17:14,ROYAL CANIN USA,35136117,FEDEX,4.41128E+11,Shipped,Delivered,CHALFONT,PA,12/28/18,12/31/18,12/31/18 0:00,1,13,US
67596,SYMSP276873,12/28/18 17:14,ROYAL CANIN USA,35136337,FEDEX,4.41128E+11,Shipped,Delivered,COCOA,FL,12/28/18,1/2/19,1/2/19 0:00,1,13,US
67597,SYMSP276874,12/28/18 17:14,ROYAL CANIN USA,14082922,FEDEX,4.41128E+11,Shipped,Delivered,IRVINE,CA,12/28/18,1/4/19,1/4/19 0:00,1,13,US
67598,SYMSP276871,12/28/18 17:14,ROYAL CANIN USA,35134300,FEDEX,4.41128E+11,Shipped,Delivered,ASBURY PARK,NJ,12/28/18,1/2/19,1/2/19 0:00,1,13,US


In [7]:
#####2018 DATASET

#update datetime format with a specified format, errors="coerce" replaces error values with NaT
# combined_2018_dataset_df["Delivered On"] = pd.to_datetime(combined_2018_dataset_df["Delivered On"], format="%m/%d/%Y", errors="coerce")
# combined_2018_dataset_df["Shipped Date"] = pd.to_datetime(combined_2018_dataset_df["Shipped Date"], format="%m/%d/%Y", errors="coerce")
combined_2018_dataset_df["Delivered On"] = pd.to_datetime(combined_2018_dataset_df["Delivered On"], errors="coerce")
combined_2018_dataset_df["Shipped Date"] = pd.to_datetime(combined_2018_dataset_df["Shipped Date"], errors="coerce")

#create "transit_time" column and calculate 
combined_2018_dataset_df["transit_time"] = combined_2018_dataset_df.apply(
    lambda row: (row["Delivered On"] - row["Shipped Date"]).days if pd.notnull(row["Delivered On"]) and pd.notnull(row["Shipped Date"]) else None,
    axis=1)

#display df
combined_2018_dataset_df.sample(5)


/var/folders/0v/_cwy54bd71x52xsydh3mz7nh0000gn/T/ipykernel_81028/2466471808.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_2018_dataset_df["Delivered On"] = pd.to_datetime(combined_2018_dataset_df["Delivered On"], errors="coerce")
/var/folders/0v/_cwy54bd71x52xsydh3mz7nh0000gn/T/ipykernel_81028/2466471808.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_2018_dataset_df["Shipped Date"] = pd.to_datetime(combined_2018_dataset_df["Shipped Date"], errors="coerce")


,Navision Order ID,Navision Created Date,Channel,Channel Order ID,Shipping Agent,Tracking No,Order Status,Status Summary,Shipping City,Shipping State,Shipped Date,Delivered On,Status,Delivered,period_number,country,transit_time
15274,SYMSP226139,3/19/18 14:13,"Amazon.com, LLC",4FDFXJZF,FEDEX,4.05187E+11,Shipped,Delivered,Kansas City,KS,2018-03-20,2018-03-22,3/22/18 6:53,1,3,US,2.0
34379,SYMSP245090,7/16/18 4:26,WisdomPanel.com,5.04257E+11,USPS,9400110200793806100069,Shipped,"Delivered, In/At Mailbox",Mount Pleasant,SC,2018-07-16,2018-07-18,7/18/18 0:00,1,8,US,2.0
28948,SYMSP239987,6/11/18 2:28,WisdomPanel.com,4.84189E+11,USPS,9405510200828762956153,Shipped,"Delivered, In/At Mailbox",Scottsdale,AZ,2018-06-11,2018-06-13,6/13/18 16:16,1,6,US,2.0
26908,SYMSP237700,5/29/18 17:13,ROYAL CANIN USA,32546300,FEDEX,7.81183E+11,Shipped,Delivered,PRIOR LAKE,MN,2018-05-30,2018-06-04,6/4/18 8:35,1,6,US,5.0
24964,SYMSP235955,5/16/18 20:24,WisdomPanel.com,4.70716E+11,USPS,9400110200828742613134,Shipped,"Delivered, To Mail Room",Austin,TX,2018-05-16,2018-05-21,5/21/18 15:45,1,5,US,5.0


In [8]:
#display transit times by shipping agent, country, period_number, count 


#####2018 DATASET

#create new column "year" and adds  year based on "Shipped Date"
combined_2018_dataset_df["year"] = combined_2018_dataset_df["Shipped Date"].dt.year

# #filter shipping agents: FedEx and USPS
# filtered_df = combined_2017_dataset_df[
#     (combined_2017_dataset_df["Shipping Agent"].isin(["USPS", "FEDEX"])) &
#     (combined_2017_dataset_df["country"] == "US")]

#filter shipping agents: USPS and in the "country" = "US"
filtered_2018_df = combined_2018_dataset_df[
    (combined_2018_dataset_df["Shipping Agent"].isin(["USPS"])) &
    (combined_2018_dataset_df["country"] == "US")]

#group by "Shipping Agent", "country" ,"year", "period_number" 
transit_time_2018_stats_df = filtered_2018_df.groupby(["Shipping Agent", "country" ,"year", "period_number"]).agg(
    #calculate the avg_transit_time 
    avg_2018_transit_time=("transit_time", "mean"), 
    #copunt the number of times transit_time calculated 
    count=("transit_time", "size")).round(2).reset_index()


transit_time_2018_stats_df

,Shipping Agent,country,year,period_number,avg_2018_transit_time,count
0,USPS,US,2018,1,3.07,4683
1,USPS,US,2018,2,3.06,4800
2,USPS,US,2018,3,2.96,3575
3,USPS,US,2018,4,2.97,3441
4,USPS,US,2018,5,2.79,4180
5,USPS,US,2018,6,2.56,4109
6,USPS,US,2018,7,2.53,3162
7,USPS,US,2018,8,2.34,3187
8,USPS,US,2018,9,2.39,3719
9,USPS,US,2018,10,2.49,3669


In [9]:
#export the df to csv
transit_time_2018_stats_df.to_csv("output/usps_transit_time_2018.csv", index=False)

In [10]:
#display transit times by shipping agent, country, period_number, count 


#####2018 DATASET

#create new column "year" and adds  year based on "Shipped Date"
combined_2018_dataset_df["year"] = combined_2018_dataset_df["Shipped Date"].dt.year

# #filter shipping agents: FedEx and USPS
# filtered_df = combined_2017_dataset_df[
#     (combined_2017_dataset_df["Shipping Agent"].isin(["USPS", "FEDEX"])) &
#     (combined_2017_dataset_df["country"] == "US")]

#filter shipping agents: USPS and in the "country" = "US"
filtered_2018_df = combined_2018_dataset_df[
    (combined_2018_dataset_df["Shipping Agent"].isin(["FEDEX"])) &
    (combined_2018_dataset_df["country"] == "US")]

#group by "Shipping Agent", "country" ,"year", "period_number" 
fedex_transit_time_2018_stats = filtered_2018_df.groupby(["Shipping Agent", "country" ,"year", "period_number"]).agg(
    #calculate the avg_transit_time 
    avg_2018_transit_time=("transit_time", "mean"), 
    #copunt the number of times transit_time calculated 
    count=("transit_time", "size")).round(2).reset_index()


fedex_transit_time_2018_stats

,Shipping Agent,country,year,period_number,avg_2018_transit_time,count
0,FEDEX,US,2018,1,3.04,656
1,FEDEX,US,2018,2,3.16,1076
2,FEDEX,US,2018,3,3.15,864
3,FEDEX,US,2018,4,2.76,782
4,FEDEX,US,2018,5,3.01,721
5,FEDEX,US,2018,6,3.38,602
6,FEDEX,US,2018,7,2.87,625
7,FEDEX,US,2018,8,2.65,549
8,FEDEX,US,2018,9,3.44,525
9,FEDEX,US,2018,10,2.74,348
